<a href="https://colab.research.google.com/github/karanm21/IITG_Research_Intern/blob/main/IITG_3sec_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wfdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import wfdb

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import posixpath
import pandas as pd

In [ ]:
wfdb.dl_database('mitdb', os.path.join(os.getcwd(), 'mitdb'))

In [ ]:
wfdb.dl_database('nsrdb', os.path.join(os.getcwd(), 'nsrdb'))

In [ ]:
from wfdb import processing
import pywt
import sys
from scipy import signal
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 15]
from sklearn import svm
# from sklearn_rvm import EMRVC
# from skrvm import RVC
from scipy.signal import find_peaks
from scipy.signal import decimate
from sklearn.decomposition import PCA

In [ ]:
records = ['mitdb/100','mitdb/101','mitdb/102', 'mitdb/103', 'mitdb/105', 'mitdb/106', 'mitdb/111', 'mitdb/112',
                 'mitdb/113','mitdb/115', 'mitdb/116', 'mitdb/118', 'mitdb/121', 'mitdb/122',  'mitdb/123', 'mitdb/124',
                 'mitdb/209','mitdb/210','mitdb/230','mitdb/231', 'mitdb/232', 'mitdb/234',
                 'mitdb/221','mitdb/222', 'mitdb/223', 'mitdb/228']

In [ ]:
pip install neurokit2

In [ ]:
pip install biosppy

In [ ]:
import neurokit2 as nk
import biosppy
from statistics import mean

In [ ]:
def startFuncMIT_V(signalString):

    signals, fields = wfdb.rdsamp(signalString)
    signals = signals[:,0]
    signals = np.array(signals, dtype=[('O', float)]).astype(float)
    signals = (signals - np.mean(signals))/(np.std(signals))
    ann = wfdb.rdann(signalString, 'atr')

    signals, ann = wfdb.processing.resample_singlechan(signals, ann, 360, 250)
    samples = ann.sample
    symbols = ann.symbol

    points = []
    pointSym = []

    for i in range(len(samples)):
        if(symbols[i] == 'V' or symbols[i] == 'F' or symbols[i] == 'E' or symbols[i] == '!'):
            points.append(samples[i])
            pointSym.append(symbols[i])

    return points, pointSym, signals, samples

In [ ]:
def startFuncMIT_N(signalString):

    signals, fields = wfdb.rdsamp(signalString)
    signals = signals[:,0]
    signals = np.array(signals, dtype=[('O', float)]).astype(float)
    signals = (signals - np.mean(signals))/(np.std(signals))
    ann = wfdb.rdann(signalString, 'atr')

    signals, ann = wfdb.processing.resample_singlechan(signals, ann, 360, 250)
    samples = ann.sample
    symbols = ann.symbol

    points = []
    pointSym = []

    for i in range(len(samples)):
        if(symbols[i] == 'N'):
            points.append(samples[i])
            pointSym.append(symbols[i])

    return points, pointSym, signals, samples

In [ ]:
def startFuncMIT_S(signalString):

    signals, fields = wfdb.rdsamp(signalString)
    signals = signals[:,0]
    signals = np.array(signals, dtype=[('O', float)]).astype(float)
    signals = (signals - np.mean(signals))/(np.std(signals))
    ann = wfdb.rdann(signalString, 'atr')

    signals, ann = wfdb.processing.resample_singlechan(signals, ann, 360, 250)
    samples = ann.sample
    symbols = ann.symbol

    points = []
    pointSym = []

    for i in range(len(samples)):
        if(symbols[i] == 'a' or symbols[i] == 'A' or symbols[i] == 'e'):
            points.append(samples[i])
            pointSym.append(symbols[i])

    return points, pointSym, signals, samples

In [ ]:
def beats(points, signals, offset, listV, listN, d):
  for i in range(len(points)):
    index= points[i]
    midindex= index-offset
    lastindex= midindex-offset

    if(i==0 and midindex<0):
        midindex=0
        lastindex=0

    elif(i==0 and midindex > 0 and lastindex < 0):
      lastindex=0;

    elif(i>0 and midindex < points[i-1]):
      midindex= points[i-1]+1
      lastindex= points[i-1]

    elif(i>0 and lastindex < points[i-1]):
      lastindex= points[i-1]+1

    a=lastindex+1
    b=midindex
    c=index

    start=b
    end=c

    while(start+d<end):
      list1= np.array([])
      list1= np.concatenate((list1, signals[start: start+d]), axis=None)
      start=start+d
      listV.append(list1)


    start=a
    end=b-1

    while(start+d<end):
        list1= np.array([])
        list1= np.concatenate((list1, signals[start: start+d]), axis=None)
        start= start+d
        listN.append(list1)


In [ ]:
def beats_N(points, signals, offset, listN, d):
  for i in range(10000, len(points), 10000):
    index= points[i]
    midindex= index-offset

    if(i==0 and midindex<0):
        midindex=0

    b=midindex
    c=index

    start=b
    end=c

    while(start+d<end):
      list1= np.array([])
      list1= np.concatenate((list1, signals[start: start+d]), axis=None)
      start=start+d
      listN.append(list1)

In [ ]:
def get_beats_V(record, listV, listN):

  points, pointSym, signals, samples = startFuncMIT_V(record)

  param=15
  fs= 250
  split=3
  offset= param*fs*60
  d= split*fs

  beats(points, signals, offset, listV, listN, d)

In [ ]:
def get_beats_S(record, listV, listN):

  points, pointSym, signals, samples = startFuncMIT_S(record)

  param=15
  fs= 250
  split=3
  offset= param*fs*60
  d= split*fs

  beats(points, signals, offset, listV, listN, d)

In [ ]:
def get_beats_N(record, listV, listN):
  points, pointSym, signals, samples = startFuncMIT_N(record)

  param=15
  fs= 250
  split=3
  offset= param*fs*60
  d= split*fs

  beats_N(points, signals, offset, listN, d)

In [ ]:
listV=[]
# listN=[]
listS=[]

for record in records:
  get_beats_V(record, listV, listN)
  get_beats_S(record, listS, listN)

In [ ]:
records_nsrdb = ['nsrdb/16265','nsrdb/16272','nsrdb/16273','nsrdb/16420','nsrdb/16483','nsrdb/16539','nsrdb/16773','nsrdb/16786','nsrdb/16795','nsrdb/17052','nsrdb/17453','nsrdb/18177','nsrdb/18184','nsrdb/19088','nsrdb/19090','nsrdb/19093','nsrdb/19140','nsrdb/19830']

In [ ]:
listN= []
for record in records_nsrdb:
  if(len(listN)>=7000):
      break
  get_beats_N(record, listN, listN)

In [ ]:
len(listV)

7097

In [ ]:
len(listS)

6371

In [ ]:
listN= listN[0:6500]

In [ ]:
len(listN)

6500

In [ ]:
listV[0].shape

(750,)

In [ ]:
listS[0].shape

(750,)

In [ ]:
listN[0].shape

(750,)

In [ ]:
# listN[0]

In [ ]:
dfV= pd.DataFrame(listV)
dfV['Type']='V'

In [ ]:
dfV.head()

,0,1,2,3,4,5,6,7,8,9,...,741,742,743,744,745,746,747,748,749,Type
0,-0.172668,-0.253933,-0.208621,-0.175584,-0.127824,-0.276282,-0.232398,-0.157888,-0.150945,-0.180683,...,-0.595638,-0.663190,-0.762077,-0.673240,-0.703165,-0.687020,-0.780980,-0.689351,-0.664274,V
1,-0.674611,-0.748461,-0.659966,-0.598095,-0.515254,-0.530925,-0.417045,-0.272807,-0.210748,-0.289992,...,-0.294144,-0.333979,-0.352597,-0.346021,-0.261900,-0.237598,-0.327143,-0.284399,-0.217052,V
2,-0.219789,-0.333529,-0.273434,-0.196434,-0.171695,-0.305003,-0.293886,-0.216530,-0.254373,-0.332083,...,0.495162,1.600858,3.376126,5.528701,6.559969,5.745043,2.902073,-0.084692,-0.959460,V
3,-0.588217,-0.319670,-0.299789,-0.278611,-0.250558,-0.322957,-0.303380,-0.254966,-0.253231,-0.310582,...,0.465407,0.267934,0.110238,0.092149,0.133804,0.079437,0.010509,-0.012704,0.052028,V
4,0.036029,-0.014221,-0.006106,0.017534,0.016153,-0.032201,0.032601,0.016340,0.056163,0.004122,...,0.428444,0.233021,0.186413,0.310939,0.362071,0.365539,0.473814,0.443541,0.282351,V


In [ ]:
dfN= pd.DataFrame(listN)
dfN['Type']= 'N'
dfN.head()

,0,1,2,3,4,5,6,7,8,9,...,741,742,743,744,745,746,747,748,749,Type
0,-0.526110,-0.420975,-0.372223,-0.287212,-0.468992,-0.259544,-0.142253,-0.140374,0.004512,-0.006137,...,-0.079503,-0.755916,-0.400573,-0.637954,-0.586329,-0.567162,-0.514607,-0.467115,-0.369239,N
1,-0.377718,-0.310403,-0.240344,-0.092023,-0.213174,-0.099317,-0.004382,0.094986,0.084632,0.095719,...,-0.484951,-1.620783,1.407230,5.139247,5.733449,2.181028,-0.751122,-0.292074,-0.559253,N
2,-0.377524,-0.489135,-0.358844,-0.441132,-0.252511,-0.314772,-0.236181,-0.197683,-0.062869,-0.062654,...,-0.241442,-0.341199,-0.209086,-0.280459,-0.256965,-0.329983,-0.166276,0.113715,0.051124,N
3,-0.140601,-0.168045,-0.271365,-0.280454,-0.361390,-0.303359,-0.373075,-0.412522,-1.815454,0.390740,...,-0.306309,-0.327800,-0.238251,-0.314638,-0.290001,-0.320235,-0.346372,-0.344971,-0.312524,N
4,-0.303220,-0.217888,-0.256114,-0.161753,0.002047,-0.173305,-0.234254,-0.270088,-0.275537,-0.354932,...,-0.362699,-0.292645,-0.429378,-0.456408,-0.457405,-0.378421,-0.314226,-0.311171,-0.194761,N


In [ ]:
dfS= pd.DataFrame(listS)
dfS['Type']= 'S'
dfS.head()

,0,1,2,3,4,5,6,7,8,9,...,741,742,743,744,745,746,747,748,749,Type
0,-0.341620,1.271093,0.567522,1.030967,0.673648,1.000170,0.826956,0.916293,0.706583,0.840706,...,0.064863,0.103441,0.114010,0.008105,0.039916,0.114787,0.160386,0.052241,0.016590,S
1,6.128223,5.057662,2.141735,-0.309836,-0.928178,-0.563843,-0.306906,-0.398673,-0.454112,-0.394014,...,0.040271,0.129069,0.217771,0.221688,0.125484,0.199818,0.294783,0.202883,0.120542,S
2,0.109496,0.147581,0.200358,0.163814,0.120544,0.161383,0.205695,0.168778,0.086907,0.117792,...,-0.358019,-0.364748,-0.276251,-0.284430,-0.381229,-0.339866,-0.246677,-0.269800,-0.339791,S
3,-0.350194,-0.238525,-0.237285,-0.323988,-0.350404,-0.244730,-0.273175,-0.389459,-0.422666,-0.271194,...,0.285938,0.378172,0.411120,0.403079,0.364467,0.380215,0.473189,0.542018,0.522361,S
4,0.544367,0.577215,0.575158,0.479415,0.415996,0.442585,0.452369,0.397839,0.290488,0.375098,...,0.138528,0.121979,0.204572,0.174916,0.096167,0.128697,0.147117,0.153399,0.091144,S


In [ ]:
from google.colab import files

In [ ]:
# dfV.to_csv('V_beats_3sec.csv', encoding = 'utf-8-sig')
dfN.to_csv('N_beats_3sec.csv', encoding = 'utf-8-sig')
# dfS.to_csv('S_beats_3sec.csv', encoding = 'utf-8-sig')

In [ ]:
# files.download('V_beats_3sec.csv')
files.download('N_beats_3sec.csv')
# files.download('S_beats_3sec.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>